In [1]:
from torchnlp.datasets import iwslt_dataset
from torchnlp.word_to_vector import GloVe
from tqdm.auto import tqdm
from utils import *
from models import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
vectors = GloVe()

In [3]:
train = [sentence['en'] for sentence in iwslt_dataset(train=True)]
dev = [sentence['en'] for sentence in iwslt_dataset(dev=True)]
test = [sentence['en'] for sentence in iwslt_dataset(test=True)]
print("Train:", len(train))
print("Dev:", len(dev))
print("Test:", len(test))

Train: 196884
Dev: 993
Test: 1305


In [4]:
print("Preprocessing the train data")
train_x, train_y = preprocess_data(train[:15000], WINDOW_SIZE, CLASSES, vectors)
print("Preprocessing the dev data")
dev_x, dev_y = preprocess_data(dev, WINDOW_SIZE, CLASSES, vectors)
print("Preprocessing the test data")
test_x, test_y = preprocess_data(test, WINDOW_SIZE, CLASSES, vectors)

Preprocessing the train data
Tokenizing:



Padding:



Creating labels:



Get word vector weights



Converting labels to tensor


...


Preprocessing the test data
Tokenizing:



Padding:



Creating labels:



Get word vector weights



Converting labels to tensor


In [5]:
train_loader = DataLoader(PuncDataset(train_x, train_y), batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=0)
dev_loader = DataLoader(PuncDataset(dev_x, dev_y), batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=0)
test_loader = DataLoader(PuncDataset(test_x, test_y), batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=0)

In [6]:
import torch.nn as nn
import torch.nn.functional as F


class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        
        self.linear = nn.Sequential(
            nn.Linear(300 * 5, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, len(CLASSES)),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

    def forward(self, data):
        flatten_data = data.reshape(data.shape[0], 1500)
        prediction = self.linear(flatten_data)
        
        return prediction

In [11]:
import pandas as pd

def train(train_loader, dev_loader, model, criterion, optimizer, epochs):

    for epoch in range(epochs):  # loop over the dataset multiple times
        print("----------------------------")
        print("Epoch:", epoch)
        
        model.train()
        t = tqdm(iter(train_loader), leave=False, total=len(train_loader))
        epoch_loss = 0.0
        
        for _, data in enumerate(t, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            epoch_loss += outputs.shape[0] * loss.item()
            
        print("Training loss:", epoch_loss / len(train_loader))
        _, acc = validate(dev_loader, model)
        print('Validation accuracy: %d %%' % (100 * acc))
                
def validate(dataloader, model):
    correct = 0
    total = 0
    predicted_total = [1] * len(CLASSES)
    predicted_correct = [0] * len(CLASSES)
    predicted_expected = [1] * len(CLASSES)
    
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            # Gathering information for f score
            for i in range(predicted.shape[0]):
                predicted_class = predicted[i]
                correct_class = labels[i]
                predicted_total[predicted_class] += 1
                predicted_expected[correct_class] += 1
                if predicted_class == correct_class:
                    predicted_correct[predicted_class] += 1
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    f_scores = []
    for i in range(len(CLASSES)):
        precision = predicted_correct[i] / predicted_total[i]
        recall = predicted_correct[i] / predicted_expected[i]
        f_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
        f_scores.append([CLASSES[i], predicted_total[i], predicted_correct[i], predicted_expected[i], precision, recall, f_score])
        
    df = pd.DataFrame(f_scores, columns=["punctuation", "predicted", "predicted correctly", "predicted expectation", "precision", "recall", "f_score"])
    df = df.set_index("punctuation")
    return df, correct / total

In [12]:
import torch.optim as optim

model = LinearModel()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

if __name__=='__main__':
    train(train_loader, dev_loader, model, criterion, optimizer, 10)

    print('Finished Training')

----------------------------
Epoch: 0


Training loss: 15.54025289388633
Validation accuracy: 92 %
----------------------------
Epoch: 1


Training loss: 15.135006038971985
Validation accuracy: 92 %
----------------------------
Epoch: 2


Training loss: 14.975052129643165
Validation accuracy: 93 %
----------------------------
Epoch: 3


Training loss: 14.860352416908949
Validation accuracy: 93 %
----------------------------
Epoch: 4


...

Training loss: 14.708072821635183
Validation accuracy: 93 %
----------------------------
Epoch: 6


Training loss: 14.68327679747512
Validation accuracy: 93 %
----------------------------
Epoch: 7


Training loss: 14.63702876048234
Validation accuracy: 93 %
----------------------------
Epoch: 8


Training loss: 14.536647459014338
Validation accuracy: 93 %
----------------------------
Epoch: 9


Training loss: 14.466326672229831
Validation accuracy: 93 %
Finished Training


In [13]:
df, acc = validate(test_loader, model)
print('Test accuracy: %d %%' % (100 * acc))
df

Test accuracy: 94 %


,predicted,predicted correctly,predicted expectation,precision,recall,f_score
punctuation,,,,,,
o,19651,18821,19072,0.957763,0.986839,0.972084
",",831,573,1355,0.689531,0.422878,0.524245
.,1314,1172,1228,0.891933,0.954397,0.922109
?,1,0,129,0.000000,0.000000,0.000000
!,1,0,14,0.000000,0.000000,0.000000


In [14]:
from datetime import datetime
now = datetime.now()

PATH = './checkpoints/linear_' + now.strftime("%m-%d-%Y-%H-%M-%S") + '.pth'
torch.save(model.state_dict(), PATH)